In [ ]:
# If needed, uncomment:
# %pip install mss opencv-python numpy ipywidgets
#pip install --upgrade pip
#pip install mss opencv-python numpy ipywidgets pynput
#jupyter nbextension enable --py widgetsnbextension


import threading, time
import numpy as np
import cv2, mss
import ipywidgets as widgets
from IPython.display import display

# --- UI: monitor dropdown, FPS slider, filename, buttons, status ---
# probe monitors once to build choices
with mss.mss() as _sct:
    _mons = _sct.monitors  # [0]=all (virtual), 1..N = each monitor

options = [("All monitors (combined)", 0)]
for i, m in enumerate(_mons[1:], start=1):
    label = f"Monitor {i} — {m['width']}x{m['height']} @ ({m['left']},{m['top']})"
    if i == 1:
        label += " [Primary]"
    options.append((label, i))

monitor_dd = widgets.Dropdown(options=options, description="Capture:")
fps_slider = widgets.IntSlider(value=30, min=5, max=60, step=1, description="FPS:")
filename = widgets.Text(value="screen.mp4", description="Output:")
start_btn = widgets.Button(description="Start", button_style="success")
stop_btn  = widgets.Button(description="Stop",  button_style="danger", disabled=True)
status    = widgets.HTML()

ui = widgets.VBox([
    widgets.HBox([monitor_dd, fps_slider]),
    filename,
    widgets.HBox([start_btn, stop_btn]),
    status
])
display(ui)

# --- recording thread machinery ---
stop_event = threading.Event()
record_thread = None

def record():
    idx = monitor_dd.value
    out_path = filename.value
    fps = int(fps_slider.value)

    with mss.mss() as sct:
        mon = sct.monitors[idx]
        w, h = mon["width"], mon["height"]

        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        writer = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

        status.value = f"<b>Recording</b> {w}x{h} @ {fps} FPS from selection {idx} → <code>{out_path}</code>"
        frame_interval = 1.0 / fps
        next_t = time.perf_counter()
        try:
            while not stop_event.is_set():
                img = sct.grab(mon)                           # BGRA
                frame = cv2.cvtColor(np.array(img), cv2.COLOR_BGRA2BGR)
                writer.write(frame)

                next_t += frame_interval
                sleep_for = next_t - time.perf_counter()
                if sleep_for > 0:
                    time.sleep(sleep_for)
        finally:
            writer.release()
            status.value = "<b>Stopped.</b> Saved to <code>{}</code>".format(out_path)

def on_start(_):
    global record_thread
    if record_thread and record_thread.is_alive():
        return
    stop_event.clear()
    start_btn.disabled = True
    stop_btn.disabled = False
    record_thread = threading.Thread(target=record, daemon=True)
    record_thread.start()

def on_stop(_):
    stop_event.set()
    stop_btn.disabled = True
    start_btn.disabled = False

start_btn.on_click(on_start)
stop_btn.on_click(on_stop)
